In [2]:
import pandas as pd

In [3]:
prot_info = pd.read_json('../data/phosphosite_sequences/phosphosite_df.json')
prot_info

,id,sites,sequence
0,A0A024R4G9,"[14, 16, 20]",MTVLEAVLEIQAITGSRLLSMVPGPARPPGSCWDPTQCTRTWLLSH...
1,A0A075B759,"[40, 79, 93, 119]",MVNSVVFFEITRDGKPLGRISIKLFADKIPKTAENFRALSTGEKGF...
2,A0A087WP46,"[359, 972, 973, 974, 988, 997, 1000, 1005, 101...",MARDGAEQPDSGPLPRPSPCPQEDRASNLMPPKPPRTWGLQLQGPS...
3,A0A087WPF7,"[32, 43, 622, 626, 798, 941, 956, 1031, 1038, ...",MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...
4,A0A087WQ53,[58],MGQNNNVTEFILLGLTQDPAGQKVLFVMFLLIYIVKIVGNLLIVGT...
...,...,...,...
42252,XP_997087,"[347, 907, 915, 918, 927]",MENFLALMNSISDTWMSPSCMDIAMDMGIAFVCGAGLFFLLLPFLK...
42253,YP_009725299,"[504, 660, 661, 794, 1826]",APTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVEL...
42254,YP_009725305,[5],NNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLS...
42255,YP_009725309,[56],AENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIP...


In [4]:
from functools import partial

In [5]:
buffer = []
def mark_fn(row, buf):
    split_seq = list(row['sequence'])
    for i in row['sites']:
        idx = eval(i) - 1
        split_seq[idx] = str.lower(split_seq[idx])
    
    for i in row['sites']:
        idx = eval(i) - 1
        left = max(0, idx - 6)
        right = min(len(split_seq), idx + 7)
        window = split_seq[left:right]
        idx = min(0, idx - 6) + 6
        window[idx] = window[idx] + '*'
        buf.append("".join(window))

prot_info.apply(partial(mark_fn, buf=buffer), axis=1)
buffer

['LEIQAIt*GsRLLs',
 'IQAItGs*RLLsMV',
 'tGsRLLs*MVPGPA',
 'ENFRALs*TGEKGF',
 'TGDKSIy*GEKFDD',
 'NLIRKHt*GSGILS',
 'FICAAKt*EWLDGK',
 'CRNGQRs*PAPELR',
 'RLGPSRs*ysVEQL',
 'LGPSRsy*sVEQLQ',
 'GPSRsys*VEQLQP',
 'ALAPQTs*TPKVKR',
 'KVKRAPs*LQtLHL',
 'RAPsLQt*LHLVsS',
 'QtLHLVs*SSHQHR',
 'QHRRATs*FQNLHS',
 'FPHRALs*VEDVGA',
 'GSGRRDs*GLYVQA',
 'SRAGLGt*GAAGGI',
 'GIGAGRt*RAPSLA',
 'VAARPGt*VPHtLL',
 'PGtVPHt*LLQKDP',
 'WNRLHRt*PPSFPT',
 'QLSRVPs*PYVRTP',
 'DSTRPNs*TSSREA',
 'PMTVGVt*GIHAMN',
 'GIHAMNs*IGSLDR',
 'DPLHRLs*tPRLYE',
 'PLHRLst*PRLYEA',
 'MHYPRIs*PTAGHQ',
 'LGGRPGs*PRRTtP',
 'GsPRRTt*PLSAEI',
 'IRERPPs*HTLKDI',
 'SLGSPMy*FFLAFL',
 'SCEREEs*LRRSRG',
 'RRSRGLs*PGRAGE',
 'VCCCRRs*CCRsCG',
 'RRsCCRs*CGCGsC',
 'RsCGCGs*CGCGCG',
 'KQARRPs*sLEsGS',
 'QARRPss*LEsGSK',
 'RPssLEs*GSKKRP',
 'SKKRPPs*FSRTRG',
 'LGGDRYt*PNFNVS',
 'SSNDRKy*ESTSRG',
 'PEVLQDs*LDRCYS',
 'PEVLQDs*LDRCYS',
 'PEVLQDs*LDRCYS',
 'PEVLQDs*LDRCYS',
 'PEVLQDs*LDRCYS',
 'PEVLQDs*LDRCYS',
 'PEVLQDs*LDRCYS',
 'PEVLQDs*LD

In [6]:
with open('../data/kinase_pred_input.txt', 'w') as f:
    f.write('\n'.join(buffer))

In [7]:
len(buffer)
for i in range(len(buffer) // 10000):
    with open(f'../data/kinase_inputs/{i}.txt', 'w') as f:
        right = min(len(buffer), (1+i) * (10000)) 
        f.write("\n".join(buffer[i * 10000: right]))


In [8]:
def mark_residue(row, residue):
    split_seq = list(row['sequence'])
    for i in row['sites']:
        idx = eval(i) - 1
        if split_seq[idx] == residue:
            split_seq[idx] = split_seq[idx] + '*'
    return "".join(split_seq)

In [9]:
marked = []
residues = ['S', 'T', 'Y']
for residue in residues:
    marked.append(prot_info.apply(lambda x: mark_residue(x, residue), axis=1))

In [10]:
for m, r in zip(marked, residues):
    m.to_csv(f'../data/marked_sites_{r}.csv', index=False)

In [11]:
import glob

dfs = []
for path in glob.glob('../data/kinase_inputs/*.tsv'):
    dfs.append(pd.read_csv(path, sep='\t'))
    

In [12]:
dfs[0]

,sequence,kinase,kinase_group,score_raw,score_log2,score_rank,site_percentile,percentile_rank
0,EIQAITGSRLLS,GAK,Other,5.311425,2.409099,1,99.597510,1
1,EIQAITGSRLLS,BIKE,Other,3.910301,1.967280,2,99.138212,2
2,EIQAITGSRLLS,AAK1,Other,3.031855,1.600201,3,98.399710,3
3,EIQAITGSRLLS,HRI,Other,1.344121,0.426663,5,94.629842,4
4,EIQAITGSRLLS,IRAK1,TKL,1.072610,0.101126,6,93.448964,5
...,...,...,...,...,...,...,...,...
320614,GLSRDSESRAIC,GRK5,AGC,2.152503,1.106015,5,95.419109,2
320615,GLSRDSESRAIC,AURA,Other,4.864564,2.282310,1,94.666103,3
320616,GLSRDSESRAIC,MOS,Other,3.153255,1.656842,2,93.650813,4
320617,GLSRDSESRAIC,AURC,Other,2.228357,1.155981,4,90.557805,5


In [13]:
thresholds = [1, 5, 10]

kinase_ranks = [None for _ in range(len(thresholds))]
group_ranks = [None for _ in range(len(thresholds))]

for df in dfs:
    for i, threshold in enumerate(thresholds[::-1]):
        temp = df[df['percentile_rank'] <= threshold].dropna()
        ki_ranks = temp.groupby('kinase')['percentile_rank'].sum()
        if kinase_ranks[i] is None:
            kinase_ranks[i] = ki_ranks
        else:
            kinase_ranks[i] = pd.Series.add(kinase_ranks[i], ki_ranks, fill_value=0)
        
        gr_ranks = temp.groupby('kinase_group')['percentile_rank'].sum()
        if group_ranks[i] is None:
            group_ranks[i] = gr_ranks
        else:
            group_ranks[i] = pd.Series.add(group_ranks[i], gr_ranks, fill_value=0)

kinase_ranks.reverse()
group_ranks.reverse()

In [14]:
n_top = 20
res = pd.DataFrame(index=range(20))
for i, tr in enumerate(thresholds):
    for rank, name in zip((kinase_ranks, group_ranks), ['kinase', 'kinase_group']):
        print(f"Top 20 {name} by top {tr} percentile rank:")
        if tr == 1:
            temp = pd.DataFrame(rank[i].sort_values(ascending=False).head(20)).reset_index()
            res[f'{name}_top{tr}'] = temp[name]
            res[f'{name}_top{tr}_sum'] = temp['percentile_rank']
            print(rank[i].sort_values(ascending=False).head(20))

        else:
            temp = pd.DataFrame(rank[i].sort_values().head(20)).reset_index()
            res[f'{name}_top{tr}'] = temp[name]
            res[f'{name}_top{tr}_sum'] = temp['percentile_rank']
            print(rank[i].sort_values().head(20))

Top 20 kinase by top 1 percentile rank:
kinase
VRK1       5114.0
AlphaK3    4704.0
CDC7       4498.0
PBK        4288.0
PINK1      4099.0
TTBK1      4058.0
SMG1       4055.0
IRAK1      3927.0
FAM20C     3830.0
CHAK2      3428.0
MEK2       3407.0
GAK        3328.0
CAMKK1     3174.0
PDHK4      3173.0
BUB1       3143.0
KIS        3046.0
PRPK       2931.0
TTK        2921.0
BIKE       2860.0
CAMKK2     2821.0
Name: percentile_rank, dtype: float64
Top 20 kinase_group by top 1 percentile rank:
kinase_group
Other    92294
CMGC     44423
CAMK     37547
TKL      33459
STE      28392
CK1      24870
AGC      19605
Alpha    11402
PIKK     10441
PDHK      7535
ACK       3884
JAK       3832
FAM20     3830
DDR       3170
TEC       2033
SRC       2003
PDGFR     1976
EPHR      1950
FRK       1722
MuSK      1251
Name: percentile_rank, dtype: int64
Top 20 kinase by top 5 percentile rank:
kinase
AKT1      1065
SGK1      1085
EPHA8     1129
AKT3      1170
AKT2      1374
MRCKB     1402
EPHB2     1614
EPHA7   

In [15]:
res

,kinase_top1,kinase_top1_sum,kinase_group_top1,kinase_group_top1_sum,kinase_top5,kinase_top5_sum,kinase_group_top5,kinase_group_top5_sum,kinase_top10,kinase_top10_sum,kinase_group_top10,kinase_group_top10_sum
0,VRK1,5114.0,Other,92294,AKT1,1065,TIER,4591,EPHA8,6418,RETR,11124
1,AlphaK3,4704.0,CMGC,44423,SGK1,1085,ROSR,4709,EPHB4,6581,TIER,11525
2,CDC7,4498.0,CAMK,37547,EPHA8,1129,RETR,4739,ZAP70,6928,ROSR,12184
3,PBK,4288.0,TKL,33459,AKT3,1170,SYK,6495,MET,7007,MuSK,14982
4,PINK1,4099.0,STE,28392,AKT2,1374,FAK,6708,SGK1,7173,SYK,15140
5,TTBK1,4058.0,CK1,24870,MRCKB,1402,FES,6746,EPHB2,7252,ABL,15569
6,SMG1,4055.0,AGC,19605,EPHB2,1614,CSK,6797,TXK,7410,FAK,17011
7,IRAK1,3927.0,Alpha,11402,EPHA7,1738,LTKR,7189,AKT1,7530,CSK,17838
8,FAM20C,3830.0,PIKK,10441,EPHB1,1748,MuSK,7220,ARG,7657,FES,17880
9,CHAK2,3428.0,PDHK,7535,EPHA2,1817,ABL,7612,EPHB1,7689,LTKR,21568


In [16]:
def rank_kinases(kinase_dfs, n_top):
    thresholds = [1, 5, 10]

    kinase_ranks = [None for _ in range(len(thresholds))]
    group_ranks = [None for _ in range(len(thresholds))]

    for df in kinase_dfs:
        for i, threshold in enumerate(thresholds[::-1]):
            temp = df[df['percentile_rank'] <= threshold].dropna()
            ki_ranks = temp.groupby('kinase')['sequence'].count()
            if kinase_ranks[i] is None:
                kinase_ranks[i] = ki_ranks
            else:
                kinase_ranks[i] = pd.Series.add(kinase_ranks[i], ki_ranks, fill_value=0)
            
            gr_ranks = temp.groupby('kinase_group')['sequence'].count()
            if group_ranks[i] is None:
                group_ranks[i] = gr_ranks
            else:
                group_ranks[i] = pd.Series.add(group_ranks[i], gr_ranks, fill_value=0)

    kinase_ranks.reverse()
    group_ranks.reverse()

    res = pd.DataFrame(index=range(n_top))
    for i, tr in enumerate(thresholds):
        for rank, name in zip((kinase_ranks, group_ranks), ['kinase', 'kinase_group']):
            temp = pd.DataFrame(rank[i].sort_values(ascending=False).head(n_top)).reset_index()
            res[f'{name}_top{tr}'] = temp[name]
            res[f'{name}_top{tr}_sum'] = temp['sequence']
            print(rank[i].sort_values(ascending=False).head(n_top))

    return res

In [17]:
def split_by_residue(residue, kinase_dfs): 
    return [df[df['sequence'].apply(lambda x: x[5] == residue)] for df in kinase_dfs]

residues = ['S', 'T', 'Y']
residue_dfs = [split_by_residue(r, dfs) for r in residues]

In [18]:
residue_dfs[2][0]

,sequence,kinase,kinase_group,score_raw,score_log2,score_rank,site_percentile,percentile_rank
151,GDKSIYGEKFDD,FGR,SRC,6.242638,2.642156,5,97.127601,1
152,GDKSIYGEKFDD,YES,SRC,7.808249,2.964999,2,96.243786,2
153,GDKSIYGEKFDD,FYN,SRC,10.805009,3.433628,1,95.838704,3
154,GDKSIYGEKFDD,SRC,SRC,6.784112,2.762160,3,94.181550,4
155,GDKSIYGEKFDD,BTK,TEC,2.588496,1.372114,6,91.566931,5
...,...,...,...,...,...,...,...,...
320376,DEEDAYERRKLE,FYN,SRC,9.326078,3.221270,6,94.641871,13
320377,DEEDAYERRKLE,CTK,CSK,4.144212,2.051098,13,94.586632,14
320378,DEEDAYERRKLE,ALK,LTKR,3.332215,1.736482,16,93.426625,15
320379,DEEDAYERRKLE,EPHA6,EPHR,3.721661,1.895947,15,92.450746,16


In [19]:
results = []
for res_dfs in residue_dfs:
    results.append(rank_kinases(res_dfs, 20))

kinase
CDC7      4486.0
TTBK1     3966.0
FAM20C    3809.0
SMG1      3770.0
PINK1     3322.0
IRAK1     3290.0
CHAK2     3208.0
PDHK4     3159.0
KIS       3041.0
PRPK      2917.0
HUNK      2686.0
ERK5      2648.0
MOS       2547.0
PDHK1     2471.0
MASTL     2449.0
ULK1      2291.0
CAMK2G    2274.0
RIPK3     2264.0
CK1D      2226.0
BUB1      2185.0
Name: sequence, dtype: float64
kinase_group
Other    65285
CMGC     36667
CAMK     33014
TKL      24445
CK1      17208
AGC      16627
STE      10406
PIKK      9660
PDHK      7490
Alpha     5604
FAM20     3809
Name: sequence, dtype: int64
kinase
TTBK1     12068.0
SMG1      11819.0
CDC7      11443.0
IRAK1     10635.0
FAM20C    10488.0
PDHK4     10031.0
PRPK       9357.0
PDHK1      9288.0
CHAK2      9141.0
ULK1       9075.0
MOS        8768.0
RIPK3      8646.0
CK2A1      8526.0
PINK1      8469.0
IRE2       8369.0
IRE1       8309.0
CK2A2      8279.0
CK1D       8168.0
CK1E       8109.0
CAMK2G     8102.0
Name: sequence, dtype: float64
kinase_group
Othe

In [20]:
results[0]

,kinase_top1,kinase_top1_sum,kinase_group_top1,kinase_group_top1_sum,kinase_top5,kinase_top5_sum,kinase_group_top5,kinase_group_top5_sum,kinase_top10,kinase_top10_sum,kinase_group_top10,kinase_group_top10_sum
0,CDC7,4486.0,Other,65285.0,TTBK1,12068.0,Other,286973.0,TTBK1,18366,Other,494629.0
1,TTBK1,3966.0,CMGC,36667.0,SMG1,11819.0,CMGC,185843.0,SMG1,16621,CAMK,350727.0
2,FAM20C,3809.0,CAMK,33014.0,CDC7,11443.0,CAMK,176989.0,IRAK1,15805,CMGC,348024.0
3,SMG1,3770.0,TKL,24445.0,IRAK1,10635.0,TKL,125936.0,CDC7,15698,TKL,242497.0
4,PINK1,3322.0,CK1,17208.0,FAM20C,10488.0,AGC,98332.0,PDHK4,15159,AGC,217783.0
5,IRAK1,3290.0,AGC,16627.0,PDHK4,10031.0,CK1,70635.0,ULK1,15121,STE,135371.0
6,CHAK2,3208.0,STE,10406.0,PRPK,9357.0,STE,61638.0,PDHK1,14765,CK1,117860.0
7,PDHK4,3159.0,PIKK,9660.0,PDHK1,9288.0,PIKK,37747.0,IRE1,14591,PIKK,57890.0
8,KIS,3041.0,PDHK,7490.0,CHAK2,9141.0,PDHK,26837.0,FAM20C,14357,PDHK,42596.0
9,PRPK,2917.0,Alpha,5604.0,ULK1,9075.0,Alpha,21160.0,PRPK,14131,Alpha,34891.0


In [21]:
results[2]

,kinase_top1,kinase_top1_sum,kinase_group_top1,kinase_group_top1_sum,kinase_top5,kinase_top5_sum,kinase_group_top5,kinase_group_top5_sum,kinase_top10,kinase_top10_sum,kinase_group_top10,kinase_group_top10_sum
0,TNK1,2758.0,ACK,3884,TNK1,4805,JAK,11644,TNK1,5451,EPHR,18237
1,DDR1,2067.0,JAK,3832,DDR1,4077,EPHR,9508,PDGFRA,4882,JAK,15386
2,JAK1,1406.0,DDR,3170,TYK2,3675,PDGFR,9280,DDR1,4762,SRC,15307
3,BRK,1356.0,TEC,2033,PDGFRA,3541,SRC,9067,TYK2,4629,PDGFR,14867
4,MUSK,1251.0,SRC,2003,JAK1,3480,ACK,7429,JAK1,4257,TEC,10902
5,TYK2,1223.0,PDGFR,1976,MUSK,3156,TEC,7274,MUSK,4161,VEGFR,9465
6,ACK,1126.0,EPHR,1950,DDR2,2988,DDR,7065,DDR2,3638,ACK,8703
7,DDR2,1103.0,FRK,1722,BRK,2643,VEGFR,5875,VEGFR3,3577,FGFR,8458
8,PDGFRA,853.0,MuSK,1251,ACK,2624,FGFR,4656,JAK2,3521,DDR,8400
9,FGR,833.0,VEGFR,1229,JAK2,2491,FRK,4613,RON,3271,TAMR,6712


In [22]:
kinase_info = pd.read_csv('../data/kinase_library/kinome_information.tsv', sep='\t')
kinase_info

,MATRIX_NAME,KINASE,GENE_NAME,TYPE,SUBTYPE,FAMILY,UNIPROT_ID,UNIPROT_ENTRY_NAME,PDB_ID,KL_LIBRARY,DUAL_SPECIFICITY,DISPLAY_NAME
0,AAK1,AAK1,AAK1,ser_thr,STK,Other,Q2M2I8,AAK1_HUMAN,4WSQ,ser_thr,False,AAK1
1,ACVR2A,ACVR2A,ACVR2A,ser_thr,STK,TKL,P27037,AVR2A_HUMAN,3Q4T,ser_thr,False,ACVR2A
2,ACVR2B,ACVR2B,ACVR2B,ser_thr,STK,TKL,Q13705,AVR2B_HUMAN,2H62,ser_thr,False,ACVR2B
3,AKT1,AKT1,AKT1,ser_thr,STK,AGC,P31749,AKT1_HUMAN,1H10,ser_thr,False,AKT1
4,AKT2,AKT2,AKT2,ser_thr,STK,AGC,P31751,AKT2_HUMAN,1GZK,ser_thr,False,AKT2
...,...,...,...,...,...,...,...,...,...,...,...,...
397,VEGFR2,VEGFR2,KDR,tyrosine,RTK,VEGFR,P35968,VGFR2_HUMAN,1VR2,tyr,False,VEGFR2
398,VEGFR3,VEGFR3,FLT4,tyrosine,RTK,VEGFR,P35916,VGFR3_HUMAN,4BSJ,tyr,False,VEGFR3
399,WEE1_TYR,WEE1,WEE1,tyrosine,ncTK,WEE,P30291,WEE1_HUMAN,1X8B,ser_thr_tyr,False,WEE1 (Tyr)
400,YES,YES,YES1,tyrosine,nRTK,SRC,P07947,YES_HUMAN,2HDA,tyr,False,YES


In [32]:
cols = ['kinase_top1', 'kinase_top5', 'kinase_top10']
kinese_index = kinase_info.set_index('KINASE')
kinase_ids = set()
for i, residue in enumerate(residues):
    top = pd.DataFrame()
    for col in cols:
        # Some kinases have two entries in the .tsv, so remove them
        ids = kinese_index.loc[results[i][col]].reset_index()['UNIPROT_ID'].drop_duplicates()
        kinase_ids.update(set(ids.to_list()))
        top[f'{col}_id'] = ids
    top.to_csv(f'../data/kinases_{residue}.csv')
    top_groups = results[i][['kinase_group_top1', 'kinase_group_top5', 'kinase_group_top5']]
    top_groups.to_csv(f'../data/kinase_groups_{residue}.csv')

In [33]:
top

,kinase_top1_id,kinase_top5_id,kinase_top10_id
0,Q13470,Q13470,Q13470
1,Q08345,Q08345,P16234
2,P23458,P29597,Q08345
3,Q13882,P16234,P29597
4,O15146,P23458,P23458
5,P29597,O15146,O15146
6,Q07912,Q16832,Q16832
7,Q16832,Q13882,P35916
8,P16234,Q07912,O60674
9,P09769,O60674,Q04912


In [57]:
kinase_ids

{'O00311',
 'O14976',
 'O15146',
 'O43318',
 'O43683',
 'O60674',
 'O75385',
 'O95382',
 'P00519',
 'P00540',
 'P07949',
 'P09769',
 'P15056',
 'P16234',
 'P23458',
 'P29597',
 'P33981',
 'P36507',
 'P36888',
 'P48730',
 'P51617',
 'P52333',
 'P57058',
 'Q02750',
 'Q02763',
 'Q04912',
 'Q05397',
 'Q06187',
 'Q07912',
 'Q08345',
 'Q13164',
 'Q13470',
 'Q13555',
 'Q13882',
 'Q15118',
 'Q15831',
 'Q16654',
 'Q16832',
 'Q2M2I8',
 'Q5S007',
 'Q5TCY1',
 'Q86Y07',
 'Q8IXL6',
 'Q8N5S9',
 'Q8TAS1',
 'Q96GX5',
 'Q96KB5',
 'Q96PY6',
 'Q96Q15',
 'Q96QP1',
 'Q96RR4',
 'Q96S44',
 'Q99683',
 'Q99986',
 'Q9BX84',
 'Q9BXM7',
 'Q9NSY1',
 'Q9UEW8',
 'Q9UF33',
 'Q9Y572'}

Input to UniProt

In [58]:
input_str = " ".join(list(kinase_ids))
input_str

'P29597 Q13555 Q99986 Q13882 Q9BX84 Q05397 O00311 Q15831 P33981 P15056 Q8IXL6 P00540 Q15118 Q08345 O60674 Q9BXM7 Q9UEW8 Q8N5S9 Q02763 P09769 Q5TCY1 O43683 P48730 O75385 O95382 Q86Y07 Q96QP1 Q9NSY1 Q9Y572 Q5S007 Q16832 Q8TAS1 P36507 P16234 P07949 Q96GX5 Q07912 Q16654 Q2M2I8 O14976 P23458 Q96Q15 P57058 Q96RR4 Q02750 Q06187 P52333 Q96S44 Q13164 P36888 Q99683 P00519 Q96KB5 Q9UF33 O15146 P51617 Q96PY6 Q13470 O43318 Q04912'

In [ ]:
from Bio import SeqIO
sequences = {}

for seq in SeqIO.parse('../data/kinases.fasta', format='fasta'):
    id = seq.id.split(sep='|')[1]
    sequences[id] = str(seq.seq)

sequences = pd.DataFrame.from_dict(sequences, orient='index', columns=['sequence'])
sequences.index.name = 'uniprot_id'
sequences.to_csv('../data/kinases.csv')

In [60]:
sequences

,sequence
uniprot_id,
P29597,MPLRHWGMARGSKPVGDGAQPMAAMGGLKVLLHWAGPGGGEPWVTF...
Q13555,MATTATCTRFTDDYQLFEELGKGAFSVVRRCVKKTSTQEYAAKIIN...
Q99986,MPRVKAAQAGRQSSAKRHLAEQFAVGEIITDMAKKEWKVGLPIGQG...
Q13882,MVSRDQAHLGPKYVGLWDFKSRTDEELSFRAGDVFHVARKEEQWWW...
Q9BX84,MKEQPVLERLQSQKSWIKGVFDKRECSTIIPSSKNPHRCTPVCQVC...
Q05397,MAAAYLDPNLNHTPNSSTKTHLGTGMERSPGAMERVLKVFHYFESN...
O00311,MEASLGIQMDEPMAFSPQRDRFQAEGSLKKNEQNFKLAGVKKDIEK...
Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...
P33981,MESEDLSGRELTIDSIMNKVRDIKNKFKNEDLTDELSLNKISADTT...
